In [1]:
print("Importing...")
import os
import torch
import torch.optim as optim
import torch.nn as nn
import os,shutil,json
import argparse
from tools.Trainer import ModelNetTrainer
from tools.split_train_test_valid import split_train_test_valid
from tools.ImgDataset import MultiviewImgDataset, SingleImgDataset
from models.MVCNN import MVCNN, SVCNN
from step_2_multiview import make_multiview_dataset

print("Done importing")

Importing...
Done importing


In [2]:
def create_folder(log_dir):
    # make summary folder
    if not os.path.exists(log_dir):
        os.mkdir(log_dir)
    else:
        print('WARNING: summary folder already exists!! It will be overwritten!!')
        shutil.rmtree(log_dir)
        os.mkdir(log_dir)

name="MVCNN"
# Batch size for the second stage,  it will be *12 images in each batch for mvcnn
bs=8
# learning rate
lr=5e-5
# weight decay
weight_decay=0.0
cnn_name="vgg11"

run_folder=os.path.dirname(os.getcwd()) + "/"
# number of views
num_views=12
# path dataset
dataset = "Test_dataset/"
# path STEP dataset
STEP_dataset = dataset + "STEP_models/"
abs_path_STEP_dataset = run_folder + "/Datasets/" + STEP_dataset
# path where to save multi views dataset
MultiViews_dataset = dataset + "MVCNN_models/"
abs_path_MultiViews_dataset = run_folder + "/Datasets/" + MultiViews_dataset
create_folder(abs_path_MultiViews_dataset)
# num train epochs each stage
num_epoch = 30
# number of classes
num_class = len(os.listdir(abs_path_STEP_dataset))
# number of models per class
num_models=len(os.listdir(abs_path_STEP_dataset + "/" + os.listdir(abs_path_STEP_dataset)[0]))

print("Num classes:", num_class)
print("Num models for class:", num_models)

pretraining = False
log_dir = name
create_folder(name)

Num classes: 2
Num models for class: 12


In [3]:
print("Making Multi View Dataset...")
MultiView_dataset = make_multiview_dataset(abs_path_STEP_dataset, abs_path_MultiViews_dataset)
print("Finished creating dataset")

print("Dividing dataset")
# divide multi views in train, validation and test set
split_train_test_valid(abs_path_MultiViews_dataset)

train_path= abs_path_MultiViews_dataset + "/*/train"
test_path= abs_path_MultiViews_dataset + "/*/test"
val_path= abs_path_MultiViews_dataset + "/*/valid"

print("Finished ")

Making Multi View Dataset...
--- Examinating: class: 0  - Model: 0_10.stp
qt-pyqt5 backend - Qt version 5.12.9
--- Examinating: class: 0  - Model: 0_11.stp
INFO:OCC.Display.backend:The qt-pyqt5 backend is already loaded...``load_backend`` can only be called once per session
qt-pyqt5 backend - Qt version 5.12.9
--- Examinating: class: 0  - Model: 0_12.stp
INFO:OCC.Display.backend:The qt-pyqt5 backend is already loaded...``load_backend`` can only be called once per session
qt-pyqt5 backend - Qt version 5.12.9
--- Examinating: class: 0  - Model: 0_13.stp
INFO:OCC.Display.backend:The qt-pyqt5 backend is already loaded...``load_backend`` can only be called once per session
qt-pyqt5 backend - Qt version 5.12.9
--- Examinating: class: 0  - Model: 0_2.stp
INFO:OCC.Display.backend:The qt-pyqt5 backend is already loaded...``load_backend`` can only be called once per session
qt-pyqt5 backend - Qt version 5.12.9
--- Examinating: class: 0  - Model: 0_3.stp
INFO:OCC.Display.backend:The qt-pyqt5 back

In [4]:
# STAGE 1
log_dir = name+'_stage_1'
create_folder(log_dir)
cnet = SVCNN(name, nclasses=num_class, pretraining=pretraining, cnn_name=cnn_name)

optimizer = optim.Adam(cnet.parameters(), lr=lr, weight_decay=weight_decay)

n_models_train = num_models * num_views

train_dataset = SingleImgDataset(train_path, scale_aug=False, rot_aug=False, num_models=n_models_train, num_views=num_views)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=bs, shuffle=True, num_workers=0)

val_dataset = SingleImgDataset(val_path, scale_aug=False, rot_aug=False, test_mode=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=bs, shuffle=False, num_workers=0)

test_dataset = SingleImgDataset(test_path, scale_aug=False, rot_aug=False, test_mode=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=bs, shuffle=False, num_workers=0)

print('num_train_files: '+str(len(train_dataset.filepaths)))
print('num_test_files: '+str(len(val_dataset.filepaths)))
print('num_validation_files: '+str(len(test_dataset.filepaths)))
trainer = ModelNetTrainer(cnet, train_loader, val_loader, optimizer, nn.CrossEntropyLoss(), 'svcnn', log_dir, num_views=1, test_loader=test_loader)
trainer.train(num_epoch)

num_train_files: 204
num_test_files: 24
num_validation_files: 48
epoch 1, step 1: train_loss 0.690; train_acc 0.375
epoch 1, step 2: train_loss 0.685; train_acc 0.750
epoch 1, step 3: train_loss 0.707; train_acc 0.500
epoch 1, step 4: train_loss 0.755; train_acc 0.250
epoch 1, step 5: train_loss 0.658; train_acc 0.750
epoch 1, step 6: train_loss 0.739; train_acc 0.375
epoch 1, step 7: train_loss 0.719; train_acc 0.250
epoch 1, step 8: train_loss 0.702; train_acc 0.500
epoch 1, step 9: train_loss 0.748; train_acc 0.250
epoch 1, step 10: train_loss 0.700; train_acc 0.500
epoch 1, step 11: train_loss 0.658; train_acc 0.875
epoch 1, step 12: train_loss 0.732; train_acc 0.250
epoch 1, step 13: train_loss 0.692; train_acc 0.375
epoch 1, step 14: train_loss 0.656; train_acc 0.625
epoch 1, step 15: train_loss 0.678; train_acc 0.625
epoch 1, step 16: train_loss 0.727; train_acc 0.250
epoch 1, step 17: train_loss 0.625; train_acc 0.875
epoch 1, step 18: train_loss 0.698; train_acc 0.375
epoch 1,

In [5]:
# STAGE 2
log_dir =name+'_stage_2'
create_folder(log_dir)
cnet_2 = MVCNN(name, cnet, nclasses=num_class, cnn_name=cnn_name, num_views=num_views)
del cnet

optimizer = optim.Adam(cnet_2.parameters(), lr=lr, weight_decay=weight_decay, betas=(0.9, 0.999))

train_dataset = MultiviewImgDataset(train_path, scale_aug=False, rot_aug=False, num_models=n_models_train, num_views=num_views)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=bs, shuffle=False, num_workers=0)# shuffle needs to be false! it's done within the trainer

val_dataset = MultiviewImgDataset(val_path, scale_aug=False, rot_aug=False, num_views=num_views)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=bs, shuffle=False, num_workers=0)

test_dataset = MultiviewImgDataset(test_path, scale_aug=False, rot_aug=False, num_views=num_views)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=bs, shuffle=False, num_workers=0)

print('num_train_files: '+str(len(train_dataset.filepaths)))
print('num_val_files: '+str(len(val_dataset.filepaths)))
trainer = ModelNetTrainer(cnet_2, train_loader, val_loader, optimizer, nn.CrossEntropyLoss(), 'mvcnn', log_dir, num_views=num_views, test_loader=test_loader)
trainer.train(num_epoch)

num_train_files: 204
num_val_files: 24
epoch 1, step 1: train_loss 0.578; train_acc 0.625
epoch 1, step 2: train_loss 0.407; train_acc 1.000
epoch 1, step 3: train_loss 0.694; train_acc 0.000
Total # of valid models:  2
val mean class acc. :  0.5
val overall acc. :  0.5
val loss :  0.7705594301223755
epoch 2, step 1: train_loss 0.436; train_acc 0.875
epoch 2, step 2: train_loss 0.400; train_acc 0.875
epoch 2, step 3: train_loss 0.225; train_acc 1.000
Total # of valid models:  2
val mean class acc. :  0.5
val overall acc. :  0.5
val loss :  0.6412848830223083
Test evaluation
Total # of test models:  4
val mean class acc. :  0.75
val overall acc. :  0.75
val loss :  0.4811704456806183
